In [409]:
import pandas as pd
import numpy as np
import xlrd

In [410]:
#read from excel
course = pd.read_excel('csv/coursefinder_1.xlsx',sheet_name='course',engine='openpyxl')
college = pd.read_excel('csv/coursefinder_1.xlsx',sheet_name='college',engine='openpyxl')
city = pd.read_excel('csv/coursefinder_1.xlsx',sheet_name='city',engine='openpyxl')
country = pd.read_excel('csv/coursefinder_1.xlsx',sheet_name='country',engine='openpyxl')


In [28]:
list_dfs= [course,college,city,country]

In [38]:
for li in list_dfs:
    li.replace(0, np.nan, inplace=True)
    li.replace("0", "", inplace=True) 

In [461]:
#to write mmodified data on excel
with pd.ExcelWriter('csv/coursefinder.xlsx') as writer:
    course.to_excel(writer,sheet_name='course')
    college.to_excel(writer,sheet_name='college')
    city.to_excel(writer,sheet_name='city')
    country.to_excel(writer,sheet_name='country')

In [90]:
course.columns[course.isnull().sum()/course.shape[0]<0.40]

Index(['course_id', 'college_id', 'course_duration', 'course_duration_value',
       'course_fees_data', 'degree_type', 'delivery_type', 'course_type',
       'course_field', 'head_short_form', 'work_experience_duration_type',
       'course_ratings', 'application_start_date', 'application_end_date',
       'program_url', 'mode', 'TOEFL', 'IELTS', 'application requirements',
       'description'],
      dtype='object')

In [92]:
college.columns[college.isnull().sum()/college.shape[0]<0.4]

Index(['college_id', 'college_name', 'colege_slug', 'college_type', 'Country',
       'Collegedunia Search Volume', 'total_students', 'college_mail_id',
       'study_abroad_city_id', 'overview', 'college_admission_data',
       'admin_rating', 'Established_Year', 'IELTS', 'TOEFL iBT', 'SAT', 'ACT',
       'total_faculty', 'Full_time_students', 'Total Students',
       'International Students', 'international students',
       'international_student', '% International Students', 'Male Students',
       'Female Students', 'Male: Female Ratio', 'Student:Faculty_Ratio',
       'students_per_faculty', 'Overall Acceptance Rate',
       'US News World Ranking', 'THE World Ranking', 'QS World Ranking',
       'CWUR rank', 'Application_Fee', 'Average (Annual)', 'hostel',
       'Hostel_mode', 'mess facilities', 'library'],
      dtype='object')

In [419]:
import math
import json

def total_fees(val):
    '''getting fees data frorm course_fees_data ,
    input json strin
    output : int
    '''
    
    fees_val =""
    if not pd.isna(val):
#         print("original data  :" ,val)
        
        try:
            fees_data = json.loads(val)


            feesData = pd.DataFrame(fees_data['all_data'][1:],columns=fees_data['all_data'][0])
#             print(feesData)
            feesData.dropna(axis=1,inplace=True)
            feesData.dropna(axis=0,inplace=True)
#             print(feesData)
    #         if list(feesData.columns).count("Total Fees")<=1:
            if len(list(feesData.columns)) == len(set(feesData.columns)):
                for col in feesData.columns[1:]:
                    feesData[col] = feesData[col].str.replace(",","")
                    feesData[col] = feesData[col].str.strip("$").replace("-",0).astype('float')
                    
#                     print(feesData[col])
                if (feesData.shape[0] >0):
                    if "Total Fees" in feesData.columns:
                        fees_val = feesData.loc[0,"Total Fees"]
                    elif "Total Fee" in  feesData.columns:
                        fees_val = feesData.loc[0,"Total Fee"]
                    else :
                        fees_val = feesData.sum(axis=1)[0]

#                     print(fees_val)

        except :
            print(val)

            
    return fees_val

In [420]:
course['total_fee_per_year'] = course['course_fees_data'].apply(total_fees)

{"all_data":[{"0":"Tution Fee","1":"Others","4":"Total Fee"},{"0":"CAD 6818.50","1":"CAD 1115","4":"CAD 7993.50"},{"0":"CAD 6818.50","1":"CAD 1115","4":"CAD 7993.50"},{"0":"CAD 6818.50","1":"CAD 1115","4":"CAD 7993.50"},{"0":"CAD 6818.50","1":"CAD 1115","4":"CAD 7993.50"},{"0":"CAD 6818.50","1":"CAD 1115","4":"CAD 7993.50"},{"0":"CAD 6818.50","1":"CAD 1115","4":"CAD 7993.50"}],"fee_type":"Term"}
{"all_data":[{"0":"Tution Fee","4":"Total Fee"},{"0":"CAD 6818.50","4":"CAD 7993.50"},{"0":"CAD 6818.50","4":"CAD 7993.50"},{"0":"CAD 6818.50","4":"CAD 7414.37"},{"0":"CAD 6818.50","4":"CAD 7993.50"},{"0":"CAD 6818.50","4":"CAD 7993.50"},{"0":"CAD 6818.50","4":"CAD 7993.50"}],"fee_type":"Term"}
{"fee_type":"Year"}
{"all_data":[["Year","Tuition Fees"],["Year 1","\u00a314,500"]],"fee_type":"Year"}


In [421]:
#making "total_fee_per_year" int type currently its object , any non int value will be  
course['total_fee_per_year'] = course['total_fee_per_year'].apply(pd.to_numeric, errors='coerce')
#check dtype of the column again 
course['total_fee_per_year'].dtypes



dtype('float64')

In [447]:
# 
print( "min value : ",course['total_fee_per_year'].min())
print( "max value : ",course['total_fee_per_year'].max())
print( "median value : ",course['total_fee_per_year'].median())
print( "mean value : ",course['total_fee_per_year'].mean())
print( "std value : ",course['total_fee_per_year'].std())

print(31250+17608)
print(31250-17608)

min value :  0.0
max value :  192400.0
median value :  27776.0
mean value :  31250.00020208466
std value :  17608.894230166643
48858
13642


In [463]:
course['fee_bucket'] = pd.Series(pd.qcut(course['total_fee_per_year'].dropna(),4,labels=False,duplicates='drop'))


In [464]:
pd.qcut(course['total_fee_per_year'].dropna(),3).unique()

[(21794.0, 32660.0], (-0.001, 21794.0], (32660.0, 192400.0]]
Categories (3, interval[float64]): [(-0.001, 21794.0] < (21794.0, 32660.0] < (32660.0, 192400.0]]

In [465]:
course['fee_bucket_range'] = pd.Series(pd.qcut(course['total_fee_per_year'].dropna(), 4,duplicates='drop'))


In [466]:
# labels_fee= [1,2,3,4,5,6]
# results, bin_edges = pd.qcut(course['total_fee_per_year'], 6,labels=[1,2,3,4,5,6],retbins=True)
# results_table = pd.DataFrame(zip(bin_edges,labels_fee),columns=['Threshold', 'Tier'])
# # 
# 
course[['fee_bucket_range','fee_bucket']].drop_duplicates()

,fee_bucket_range,fee_bucket
0,"(27776.0, 35584.0]",2.0
1,"(19646.0, 27776.0]",1.0
38,"(-0.001, 19646.0]",0.0
201,"(35584.0, 192400.0]",3.0
312,NaN,NaN


In [456]:
datafile = course.join(college,on ="college_id",how='left',lsuffix='_course',rsuffix='_college')

In [457]:

user_stream = "Civil Engineering"
user_country = "usa"
user_level="Bachelor"
stream_filter = datafile['course_field']==user_stream
country_filter = datafile['Country']==user_country
level_filter = datafile['degree_type']==user_level

filtered_data = datafile[stream_filter&level_filter]

In [458]:
filtered_data[['college_name','course_field','course_duration','course_duration_value', 'degree_type',
       'delivery_type', 'course_type', 'head_short_form', 'work_experience', 'program_url', 'mode',
       'TOEFL', 'IELTS_course', 'GRE_course', 'PTE_course', 'SAT_course',
       'MELAB_course', 'MCAT_course', 'ACT_course', 'TOEIC',
       'application requirements', 'Established_Year',
        'IELTS_college', 'TOEFL_PBT', 'TOEFL_iBT', 'TOEFL_CBT',
       'CAEL' ,'MELAB_college', 'PTE_college', 'SAT_college', 'GRE_college', 'GMAT',
       'ACT_college', 'LSAT', 'MCAT_college', 
       'US News World Ranking', 'THE World Ranking', 'QS World Ranking',
       'CWUR rank', 'AWUR Rank', 'RUR Rank', 'Total_Citations',
       'Application_Fee','total_fee_per_year','fee_bucket_range','fee_bucket']].head(5)

,college_name,course_field,course_duration,course_duration_value,degree_type,delivery_type,course_type,head_short_form,work_experience,program_url,...,THE World Ranking,QS World Ranking,CWUR rank,AWUR Rank,RUR Rank,Total_Citations,Application_Fee,total_fee_per_year,fee_bucket_range,fee_bucket
6,University of Alberta,Civil Engineering,year,4,Bachelor,full_time,Bachelor of Engineering [B.Eng],B.Eng,NaN,https://admissions.carleton.ca/programs/civil-...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32401.0,"(21794.0, 32660.0]",1.0
117,Queen's University,Civil Engineering,year,4,Bachelor,full_time,Bachelor of Science [B.Sc],B.Sc,NaN,https://apps.admissions.ualberta.ca/programs/e...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31852.0,"(21794.0, 32660.0]",1.0
259,Ryerson University,Civil Engineering,year,4,Bachelor,full_time,Bachelor of Science [B.Sc],NaN,NaN,https://live-ucalgary.ucalgary.ca/future-stude...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18136.0,"(-0.001, 21794.0]",0.0
433,University of British Columbia,Civil Engineering,year,4,Bachelor,full_time,Bachelor of Engineering [B.Eng],B.Eng,NaN,http://learn.uwindsor.ca/civil-engineering?elq...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39214.0,"(32660.0, 192400.0]",2.0
434,University of British Columbia,Civil Engineering,year,4,Bachelor,full_time,Bachelor of Applied Science [B.ASc],B.ASc,NaN,http://www.uwindsor.ca/studentrecruitment/339/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39214.0,"(32660.0, 192400.0]",2.0


In [ ]:
#Additon Filters


In [408]:
filtered_data[['TOEFL', 'IELTS_course', 'GRE_course', 'PTE_course', 'SAT_course',
       'MELAB_course', 'MCAT_course', 'ACT_course', 'TOEIC']].dtypes

TOEFL           float64
IELTS_course    float64
GRE_course      float64
PTE_course      float64
SAT_course      float64
MELAB_course    float64
MCAT_course     float64
ACT_course      float64
TOEIC           float64
dtype: object

In [434]:
country.columns
country.head()

,Country,student visa documents,chance of losing job,Job security rank,ease of visa approval,work visa requirements,work environment,average income,top companies,lifestyle and living condition,highest paying sector,secor with highest no. of jobs,insurance and providence at workplace,various allowances
0,canada,WORK AUTHORIZATION FOR STUDENTS:\nStudent Appl...,0.066,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,australia,WORK AUTHORIZATION FOR STUDENTS:\nStudent Appl...,0.044,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,germany,WORK AUTHORIZATION FOR STUDENTS\nStudent Appli...,0.032,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ireland,WORK AUTHORIZATION FOR STUDENTS:\nEligibility ...,0.064,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,new zealand,Types of Student Visas In New Zealand\nThere a...,0.058,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [433]:
# import spacy
import spacy

# load english language model
nlp = spacy.load('en_core_web_sm',disable=['ner','textcat'])

In [439]:
country.columns = [x.strip() for x in country.columns]

In [441]:
text = country.loc[0,'student visa documents']
doc = nlp(text)

In [442]:
for token in doc:
    print(token.text,'->',token.pos_)

WORK -> VERB
AUTHORIZATION -> NOUN
FOR -> ADP
STUDENTS -> NOUN
: -> PUNCT

 -> SPACE
Student -> PROPN
Applicant -> PROPN
: -> PUNCT


 -> SPACE
Students -> NOUN
must -> AUX
be -> VERB
over -> ADP
18 -> NUM
years -> NOUN
of -> ADP
age -> NOUN

 -> SPACE
International -> PROPN
Students -> PROPN
can -> AUX
work -> VERB
on -> ADP
- -> PUNCT
campus -> NOUN
or -> CCONJ
off -> ADP
- -> PUNCT
campus -> NOUN
for -> ADP
up -> ADP
to -> PART
20 -> NUM
hours -> NOUN
during -> ADP
university -> NOUN
semesters -> NOUN
and -> CCONJ
full -> ADJ
- -> PUNCT
time -> NOUN
during -> ADP
breaks -> NOUN
such -> ADJ
as -> ADP
the -> DET
winter -> NOUN
or -> CCONJ
summer -> NOUN
holidays -> NOUN
, -> PUNCT
without -> ADP
the -> DET
need -> NOUN
for -> ADP
a -> DET
work -> NOUN
permit -> NOUN

 -> SPACE
Foreign -> ADJ
students -> NOUN
who -> PRON
are -> AUX
studying -> VERB
in -> ADP
a -> DET
program -> NOUN
that -> DET
requires -> VERB
work -> NOUN
experience -> NOUN
such -> ADJ
as -> ADP
through -> ADP
a -> D

IRCC -> PROPN
) -> PUNCT
operates -> VERB
a -> DET
Post -> PROPN
- -> ADJ
Graduate -> ADJ
Work -> PROPN
Permit -> PROPN
Program -> PROPN
. -> PUNCT
Under -> ADP
this -> DET
program -> NOUN
, -> PUNCT
international -> ADJ
graduates -> NOUN
may -> AUX
obtain -> VERB
an -> DET
open -> ADJ
work -> NOUN
permit -> NOUN
for -> ADP
up -> ADP
to -> PART
three -> NUM
years -> NOUN

 -> SPACE
This -> DET
allows -> VERB
them -> PRON
to -> PART
work -> VERB
for -> ADP
any -> DET
Canadian -> ADJ
employer -> NOUN
in -> ADP
any -> DET
industry -> NOUN
without -> ADP
the -> DET
requirement -> NOUN
of -> ADP
a -> DET
Canadian -> ADJ
job -> NOUN
offer -> NOUN
at -> ADP
the -> DET
time -> NOUN
of -> ADP
applying -> VERB


In [443]:
from spacy import displacy 
displacy.render(doc, style='dep',jupyter=True)